# Libraries

In [23]:
import numpy as np
import pandas as pd

from sklearn.cluster import AgglomerativeClustering, DBSCAN, Birch, KMeans
from sklearn.metrics import silhouette_score

# Data

In [2]:
file_path = "/Users/catherinepollack/Documents/dartmouth/research/aim3_facebook_covid19_obesity/data"
dat_obesity = "/211105_feature_matrix_bert.csv"
dat_health = "/211118_feature_matrix_bert_health_comparator.csv"
dat_nonhealth = "/211118_feature_matrix_bert_nonhealth_comparator.csv"

dat_obesity = pd.read_csv(str(file_path + dat_obesity))
dat_health = pd.read_csv(str(file_path + dat_health))
dat_nonhealth = pd.read_csv(str(file_path + dat_nonhealth))

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (84) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (501) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Feature Matrix Curation

## Obesity

In [3]:
#api whatsapp : without skip
#neg: compound
#WC: OtherP
#BERT 0 : BERT 767
dat_obesity_1 = dat_obesity.loc[:,"api whatsapp":"without skip"]
dat_obesity_2 = dat_obesity.loc[:,"neg":"compound"]
dat_obesity_3 = dat_obesity.loc[:,"WC":"OtherP"]
dat_obesity_4 = dat_obesity.loc[:,"BERT 0":"BERT 767"]
dat_obesity_fm = pd.concat([dat_obesity_1,
                            dat_obesity_2,
                            dat_obesity_3,
                            dat_obesity_4],
                            axis = 1)
dat_obesity_fm.head()                           


,api whatsapp,arm reduce,belly fat,blood pressure,body fat,body weight,childhood obesity,com url,detail whatsapp,diabetes heart,...,BERT 758,BERT 759,BERT 760,BERT 761,BERT 762,BERT 763,BERT 764,BERT 765,BERT 766,BERT 767
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.219545,-0.014961,-0.064851,-0.038482,0.084405,0.075298,0.065782,0.248788,0.091218,0.086831
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.034646,-0.170634,-0.160933,0.004613,0.014560,-0.077150,0.029200,0.442995,0.137200,0.112337
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005560,-0.182919,-0.178136,0.028645,0.152068,0.056867,0.086266,-0.014318,0.170697,0.212658
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.242043,0.114247,-0.133007,0.009129,0.252129,-0.073247,0.107698,0.175957,0.345249,0.112090
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.256913,-0.102631,-0.296323,-0.131713,0.172565,0.027309,-0.172738,0.040497,0.369843,0.041904


## Health Comparator

In [4]:
#api whatsapp : without skip
#neg: compound
#WC: OtherP
#BERT 0 : BERT 767
dat_health_1 = dat_health.loc[:,"abdominal pain":"youtube channel"]
dat_health_2 = dat_health.loc[:,"neg":"compound"]
dat_health_3 = dat_health.loc[:,"WC":"OtherP"]
dat_health_4 = dat_health.loc[:,"BERT 0":"BERT 767"]
dat_health_fm = pd.concat([dat_health_1,
                            dat_health_2,
                            dat_health_3,
                            dat_health_4],
                            axis = 1)
                      


## Non-Health Comparator

In [5]:
#api whatsapp : without skip
#neg: compound
#WC: OtherP
#BERT 0 : BERT 767
dat_nonhealth_1 = dat_nonhealth.loc[:,"ago father":"zfat mix"]
dat_nonhealth_2 = dat_nonhealth.loc[:,"neg":"compound"]
dat_nonhealth_3 = dat_nonhealth.loc[:,"WC":"OtherP"]
dat_nonhealth_4 = dat_nonhealth.loc[:,"BERT 0":"BERT 767"]
dat_nonhealth_fm = pd.concat([dat_nonhealth_1,
                            dat_nonhealth_2,
                            dat_nonhealth_3,
                            dat_nonhealth_4],
                            axis = 1)

# Clustering

- Agglomerative Clustering
- DBScan
- KMeans
- BIRCH
- Gaussian Mixture Model

## Non-Health Comparator

In [12]:
print("Agglomerative")
model_agg = AgglomerativeClustering()
yhat_agg = model_agg.fit_predict(dat_nonhealth_fm)

print("DBSCAN")
model_db = DBSCAN()
yhat_db = model_db.fit_predict(dat_nonhealth_fm)

print("BIRCH")
model_birch= Birch()
yhat_birch = model_birch.fit_predict(dat_nonhealth_fm)

print("KMeans")
model_kmeans = KMeans()
yhat_kmeans = model_kmeans.fit_predict(dat_nonhealth_fm)


Agglomerative
DBSCAN
BIRCH
KMeans
Gaussian


In [20]:
nonhealth_labels = pd.DataFrame({"agglomerate": yhat_agg,
                    "dbscan": yhat_db,
                    "birch": yhat_birch,
                    "kmeans": yhat_kmeans})

In [21]:
print("Silhouette Scores for Nonhealth Comparator")
print("Agglomerative")
print(silhouette_score(dat_nonhealth_fm, nonhealth_labels.agglomerate, metric='euclidean')) #0.358
print("DBSCAN")
print(silhouette_score(dat_nonhealth_fm, nonhealth_labels.dbscan, metric='euclidean')) #0.106
print("BIRCH")
print(silhouette_score(dat_nonhealth_fm, nonhealth_labels.birch, metric='euclidean')) #0.343
print("KMeans")
print(silhouette_score(dat_nonhealth_fm, nonhealth_labels.kmeans, metric='euclidean')) #0.241

,agglomerate,dbscan,birch,kmeans,gmm
0,0,0,0,0,0
1,0,1,0,2,0
2,0,0,0,0,0
3,0,1,0,2,0
4,0,-1,0,5,0


## Health Comparator

In [ ]:
print("Agglomerative")
model_agg = AgglomerativeClustering()
yhat_agg = model_agg.fit_predict(dat_health_fm)

print("DBSCAN")
model_db = DBSCAN()
yhat_db = model_db.fit_predict(dat_health_fm)

print("BIRCH")
model_birch= Birch()
yhat_birch = model_birch.fit_predict(dat_health_fm)

print("KMeans")
model_kmeans = KMeans()
yhat_kmeans = model_kmeans.fit_predict(dat_health_fm)

health_labels = pd.DataFrame({"agglomerate": yhat_agg,
                    "dbscan": yhat_db,
                    "birch": yhat_birch,
                    "kmeans": yhat_kmeans})

print("Silhouette Scores for Health Comparator")
print("Agglomerative")
print(silhouette_score(dat_health_fm, health_labels.agglomerate, metric='euclidean')) #0.358
print("DBSCAN")
print(silhouette_score(dat_health_fm, health_labels.dbscan, metric='euclidean')) #0.106
print("BIRCH")
print(silhouette_score(dat_health_fm, health_labels.birch, metric='euclidean')) #0.343
print("KMeans")
print(silhouette_score(dat_health_fm, health_labels.kmeans, metric='euclidean')) #0.241

## Obesity Data

In [ ]:
print("Agglomerative")
model_agg = AgglomerativeClustering()
yhat_agg = model_agg.fit_predict(dat_obesity_fm)

print("DBSCAN")
model_db = DBSCAN()
yhat_db = model_db.fit_predict(dat_obesity_fm)

print("BIRCH")
model_birch= Birch()
yhat_birch = model_birch.fit_predict(dat_obesity_fm)

print("KMeans")
model_kmeans = KMeans()
yhat_kmeans = model_kmeans.fit_predict(dat_obesity_fm)

obesity_labels = pd.DataFrame({"agglomerate": yhat_agg,
                    "dbscan": yhat_db,
                    "birch": yhat_birch,
                    "kmeans": yhat_kmeans})

print("Silhouette Scores for Health Comparator")
print("Agglomerative")
print(silhouette_score(dat_obesity_fm, obesity_labels.agglomerate, metric='euclidean')) #0.358
print("DBSCAN")
print(silhouette_score(dat_obesity_fm, obesity_labels.dbscan, metric='euclidean')) #0.106
print("BIRCH")
print(silhouette_score(dat_obesity_fm, obesity_labels.birch, metric='euclidean')) #0.343
print("KMeans")
print(silhouette_score(dat_obesity_fm, obesity_labels.kmeans, metric='euclidean')) #0.241